In [ ]:
from utils.ratpolys import ratpoly_inv
from typing import Sequence, TypeVar, Union as U
from utils.ctrl import siso_numden
from control import TransferFunction
from control import matlab as m

T = TypeVar("T")
Poly = Sequence[T]
RatPoly = tuple[Poly[T], Poly[T]]

def osap(num: Poly[T], den: Poly[T]
) -> tuple[RatPoly[U[T, int]],
           RatPoly[U[T, int]],
           RatPoly[U[T, int]]]:
    """
    Not tested
    """
    inum, iden = ratpoly_inv(num, den)
    b0, *b_rest = iden
    a_n = inum
    controller = ratpoly_inv([1], a_n)
    feedback = ratpoly_inv(b_rest, [1])
    return ([b0, 0], [1]), controller, feedback

def osap_sys(plant: TransferFunction):
    dt = plant.dt
    return tuple(
        TransferFunction(*numden, dt)
        for numden in osap(*siso_numden(plant.minreal())))

def osap_cl(plant: TransferFunction) -> TransferFunction:
    pre, controller, feedback = osap_sys(plant)
    return pre*m.feedback(controller*plant, feedback)